# DATA 620 Final Project

## 1. Importing Libaries

In [16]:
import requests
import json
from datetime import datetime, timedelta
import time
from concurrent.futures import ThreadPoolExecutor
import os
import json

## 2. Setting up the data Pull

### 2.1 Creating a process to get all Reviews

We'll start this by creating a function that'll pull 100 reviews at a time and then creating a second function that'll allow us to pull these reviews in parallel.

In [6]:
def fetch_review_page(appid, cursor, start_date_timestamp, num_per_page, review_type, language):
    """
    Helper function to fetch a single page of Steam reviews for concurrent execution.
    Returns a tuple of (reviews, next_cursor, should_stop).
    """
    url = f"https://store.steampowered.com/appreviews/{appid}"
    params = {
        'json': 1,
        'filter': 'recent',
        'language': language,
        'review_type': review_type,
        'num_per_page': num_per_page,
        'cursor': cursor
    }

    try:
        # Note: We do not include time.sleep(1) here yet. 
        # The main function handles the rate limiting across all requests.
        response = requests.get(url, params=params, stream=True)
        response.raise_for_status()
        data = response.json()

        if data.get('success') != 1:
            return ([], None, True) # API call failed

        reviews = data.get('reviews', [])
        next_cursor = data.get('cursor')
        
        filtered_reviews = []
        should_stop = False

        for review in reviews:
            review_timestamp = review.get('timestamp_created')
            
            # Stop if we find a review older than the target date
            if review_timestamp and review_timestamp < start_date_timestamp:
                should_stop = True
                break
            
            filtered_reviews.append(review)
        
        return (filtered_reviews, next_cursor, should_stop)

    except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
        print(f"An error occurred fetching cursor {cursor[:10]}: {e}")
        return ([], None, True)

In [7]:
def get_steam_reviews_from_2025(appid, num_per_page=100, review_type='all', language='english', max_workers=5):
    """
    Retrieves all user reviews for a Steam App ID posted from January 1st, 2025 onward,
    using concurrent requests.
    """
    all_reviews = []
    
    # Define a fixed start date timestamp (January 1st, 2025)
    start_date = datetime(2025, 1, 1)
    start_date_timestamp = start_date.timestamp()

    print(f"Starting to fetch reviews for App ID {appid}. Looking for reviews newer than {start_date.strftime('%Y-%m-%d')}.")

    # --- NEW CONCURRENCY LOGIC STARTS HERE ---
    
    # Cursors to be fetched. Start with the initial cursor '*'.
    cursors_to_fetch = ['*']
    
    # Cursors that are currently being processed or scheduled.
    scheduled_cursors = set()
    
    # We will use a flag to signal when we've hit the chronological limit
    stop_fetching = False

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        
        futures = {} # Maps future objects to the cursor they are processing

        while cursors_to_fetch or futures:
            
            # 1. Schedule new jobs up to the max_workers limit
            while cursors_to_fetch and len(futures) < max_workers:
                cursor = cursors_to_fetch.pop(0)
                if cursor not in scheduled_cursors:
                    print(f"Scheduling page with cursor: {cursor[:10]}...")
                    future = executor.submit(
                        fetch_review_page, 
                        appid, cursor, start_date_timestamp, num_per_page, review_type, language
                    )
                    futures[future] = cursor
                    scheduled_cursors.add(cursor) # Mark as scheduled

            # If no jobs are scheduled and none are running, break
            if not futures:
                break
                
            # 2. Process results as they complete
            completed_futures = []
            for future in futures:
                if future.done():
                    try:
                        reviews, next_cursor, should_stop_page = future.result()
                        
                        if should_stop_page:
                            stop_fetching = True
                            
                        # Only append if we haven't hit the overall stop flag
                        if not stop_fetching:
                            all_reviews.extend(reviews)
                            
                        # If a next cursor is provided, and we're not stopping, add it to the queue
                        if next_cursor and not stop_fetching and next_cursor not in scheduled_cursors:
                            cursors_to_fetch.append(next_cursor)
                        
                        print(f"Page processed. Total reviews: {len(all_reviews)}. Next cursor: {next_cursor[:10] if next_cursor else 'None'}. Stop flag: {should_stop_page}")

                    except Exception as e:
                        print(f"Job failed for cursor {futures[future][:10]}: {e}")
                    
                    completed_futures.append(future)
            
            # Remove completed jobs from the dictionary
            for future in completed_futures:
                del futures[future]

            if stop_fetching:
                print("Chronological limit reached by a worker. Shutting down executor.")
                # Cancel pending jobs and stop scheduling new ones
                for future in futures:
                    future.cancel()
                break

            # Add a small delay to avoid busy-waiting, but be mindful of Steam's rate limit
            # Note: Parallel fetching already means higher rate, so the inherent delay 
            # while waiting for results helps, but we might need a more controlled throttling 
            # if this were a production system.
            time.sleep(0.5)

    return all_reviews

### 2.2 Creating a process to get all Steam App IDs

In [12]:
def get_all_steam_appids():
    """
    Fetches the list of all Steam applications and their App IDs using the
    GetAppList API endpoint.
    
    Returns:
        list: A list of dictionaries, where each dict has keys 'appid' and 'name'.
        Returns an empty list if the API call fails.
    """
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    
    print("🚀 Fetching complete list of all Steam applications. This may take a moment...")
    
    try:
        # Steam's GetAppList is a public, unauthenticated endpoint
        response = requests.get(url, timeout=30)
        response.raise_for_status() # Raise an exception for bad status codes
        
        data = response.json()
        
        # Navigate the JSON structure to the list of apps
        apps = data.get('applist', {}).get('apps', [])
        
        print(f"✅ Successfully retrieved {len(apps):,} applications.")
        return apps
        
    except requests.exceptions.RequestException as e:
        print(f"❌ API request failed: {e}")
        return []
    except json.JSONDecodeError:
        print("❌ Failed to decode JSON response.")
        return []

## 3. Retrieve Steam Reviews
### 3.1 Getting all Steam App IDs

In [19]:
for steam_app in get_all_steam_appids():
    save_filename = f"{steam_app['appid']}.json"
    save_filepath = f"data/{save_filename}"
    
    if os.path.exists(save_filepath):
        print(f'Game "{steam_app['name']}" data already downloaded. Skipping.')
    
    else:
        reviews = get_steam_reviews_from_2025(
            appid=steam_app['appid'],
            num_per_page=100,
            review_type='all',
            language='english'
        )

        with open(save_filepath, 'w', encoding='utf-8') as f:
            json.dump(reviews, f, ensure_ascii=False, indent=4)

🚀 Fetching complete list of all Steam applications. This may take a moment...


✅ Successfully retrieved 276,345 applications.
Starting to fetch reviews for App ID 5. Looking for reviews newer than 2025-01-01.
Scheduling page with cursor: *...
Page processed. Total reviews: 0. Next cursor: *. Stop flag: False
Starting to fetch reviews for App ID 7. Looking for reviews newer than 2025-01-01.
Scheduling page with cursor: *...
Page processed. Total reviews: 0. Next cursor: *. Stop flag: False
Starting to fetch reviews for App ID 8. Looking for reviews newer than 2025-01-01.
Scheduling page with cursor: *...
Page processed. Total reviews: 0. Next cursor: *. Stop flag: False
Starting to fetch reviews for App ID 10. Looking for reviews newer than 2025-01-01.
Scheduling page with cursor: *...
Page processed. Total reviews: 100. Next cursor: AoJ4i5rI+5. Stop flag: False
Scheduling page with cursor: AoJ4i5rI+5...
Page processed. Total reviews: 200. Next cursor: AoJww8qb0p. Stop flag: False
Scheduling page with cursor: AoJww8qb0p...
Page processed. Total reviews: 300. Next 

KeyboardInterrupt: 

## 4. Sentiment Analysis

### 4.1 Data Prep & Text Mining

### 4.2 Analysis

## 5. Network Analysis for Reviewers

### 5.1 Network Construction

### 5.2 Core Network Analysis

In [5]:
print("\n--- Final Results Summary ---")
print(f"Total English reviews from the last year: **{len(recent_reviews)}**")

if recent_reviews:
    # Check the timestamp of the newest and oldest found review
    newest_ts = max(r['timestamp_created'] for r in recent_reviews)
    oldest_ts = min(r['timestamp_created'] for r in recent_reviews)
    
    print(f"Newest Review Timestamp: {datetime.fromtimestamp(newest_ts).strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Oldest Review Timestamp: {datetime.fromtimestamp(oldest_ts).strftime('%Y-%m-%d %H:%M:%S')}")


--- Final Results Summary ---
Total English reviews from the last year: **99**
Newest Review Timestamp: 2025-11-08 12:58:02
Oldest Review Timestamp: 2025-11-08 07:37:14


{'recommendationid': '208734320',
 'author': {'steamid': '76561199089646276',
  'num_games_owned': 0,
  'num_reviews': 2,
  'playtime_forever': 16372,
  'playtime_last_two_weeks': 1606,
  'playtime_at_review': 16372,
  'last_played': 1762623802},
 'language': 'english',
 'review': 'worst game ever.. love it',
 'timestamp_created': 1762623822,
 'timestamp_updated': 1762623822,
 'voted_up': True,
 'votes_up': 0,
 'votes_funny': 0,
 'weighted_vote_score': 0.5,
 'comment_count': 0,
 'steam_purchase': True,
 'received_for_free': False,
 'written_during_early_access': False,
 'primarily_steam_deck': False}